In [1]:
#!pip install selenium
# !pip install bs4

In [2]:
from urllib import request
from selenium import webdriver
import pandas as pd
import numpy as np
import bs4
import requests
import re
from time import sleep

In [3]:
print ("BeautifulSoup version:%6.6s" % bs4.__version__)

BeautifulSoup version: 4.6.0


# 스타벅스 상점명 크롤링

In [4]:
# 다운받은 chromedriver의 path설정과 크롤링주소 설정
path=r"C:\Users\Kavin\Downloads\chromedriver_win32\chromedriver"
driver=webdriver.Chrome(path)
driver.get("https://www.istarbucks.co.kr/store/store_map.do")
sleep(5)

In [5]:
# 스타벅스 홈페이지 내에서 
location_search = driver.find_element_by_class_name('loca_search')
location_search.click()
sleep(5)

city_name_list = [el.text for el in driver.find_elements_by_xpath("//*[@id='container']/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li")]
city_name_list

['서울',
 '경기',
 '광주',
 '대구',
 '대전',
 '부산',
 '울산',
 '인천',
 '강원',
 '경남',
 '경북',
 '전남',
 '전북',
 '충남',
 '충북',
 '제주',
 '세종']

### #1 세종시 제외한 16개 시/도 크롤링

In [6]:
# test코드(수정 요망)
i = 0
while i < len(city_name_list)-1:
    # 지역선택 클릭하기
    location_search = driver.find_element_by_class_name('loca_search')
    location_search.click()
    sleep(5)
    
    # 시 선택
    city= driver.find_element_by_class_name('sido_arae_box')
    city_li = city.find_elements_by_tag_name('li')
    city_li[i].click()
    sleep(5)
    
    #구군에서 '전체' 선택하도록 하기
    gugun = driver.find_element_by_class_name('gugun_arae_box')
    gu_li = gugun.find_element_by_tag_name('li')
    gu_li.click()
    sleep(5)

    source = driver.page_source
    bs = bs4.BeautifulSoup(source,'lxml')
    entire = bs.find('ul', class_='quickSearchResultBoxSidoGugun')
    li_list = entire.find_all('li')
    
    name_list = []
    address_list = []

    # DT점명 가져오기
    for name in li_list:
        name_list.append(name.find('strong').text)

    # DT주소 가져오기
    for address in li_list:
        address_list.append(address.find('p').text)
        
        
    # csv파일 저장하기 - path, filename 설정(지역별로 파일명 상이)
    path = "C:/dataitgirls/" #저장하고 싶은 경로 앞부분 
    full_path = path + 'starbucks_' + city_name_list[i] +'.csv' #현재 경로 + 파일명
            
    df = pd.DataFrame({'name':name_list, 'address':address_list})
    
    # 'address' 칼럼에 전화번호 제거
    df['address'] = df['address'].str.replace(r'\d{2,3}-\d{3,4}-\d{4}', '')
    
    # csv파일로 저장
    df.to_csv(full_path, header=True, index=True, encoding='euc-kr')
    
    i = i+1

### #2 세종시 크롤링

In [7]:
# 세종시는 구/군 선택이 없으므로 while문 반복 이후 따로 크롤링

# 지역선택 클릭하기
loca = driver.find_element_by_class_name('loca_search')
loca.click()
sleep(5)
    
# 시 선택
loca = driver.find_element_by_class_name('sido_arae_box')
li = loca.find_elements_by_tag_name('li')
li[16].click()
sleep(5)

source = driver.page_source
bs = bs4.BeautifulSoup(source,'lxml')
entire = bs.find('ul', class_='quickSearchResultBoxSidoGugun')
li_list = entire.find_all('li')

name_list = []
address_list = []

# DT점명 가져오기
for name in li_list:
    name_list.append(name.find('strong').text)

# DT주소 가져오기
for address in li_list:
    address_list.append(address.find('p').text)
    
df_Sejong = pd.DataFrame({'name':name_list, 'address':address_list})

# 'address' 칼럼에 전화번호 제거
df_Sejong['address'] = df_Sejong['address'].str.replace(r'\d{2,3}-\d{3,4}-\d{4}', '')

# csv파일로 저장
df_Sejong.to_csv("C:/dataitgirls/starbucks_세종.csv", header=True, index=True, encoding='euc-kr')